In [1]:
import numpy as np
import pandas as pd
import json
import os
import requests

import sys
sys.path.insert(0, "/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood")

from settings import GLOBAL_NAME_DICT, start_urls_dict, dp_name_dict
from functions import *
import scrapy
from datetime import datetime

from scrapy.crawler import CrawlerProcess
from scrapy.utils.log import configure_logging
import logging
from imp import reload
reload(logging)

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time

#3 to do: checks on clean data: final >>> initial; final<initial; first place but no first price, second place not second price
##to do: protocols: how to extract a table, somwhat difficult with regex expression 

# import tkinter
# import camelot
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.CRITICAL)

# name = '/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood/table_format/exports/protocols/Protokol+8-1-2023.pdf'

# abc = camelot.read_pdf(name)   #address of file location
 
# print(abc[0].df)
#reads table from pdf file
# df = wrapper.read_pdf(name,pages="all") #address of pdf file
# print(tabulate(df))
#from ctypes.util import find_library
#find_library("gs")

#---------------------------------------------

filename = f'{os.path.join(os.getcwd())}/logging.txt'
logging.basicConfig(filename=filename,
                    filemode='a',
#                     encoding='utf-8',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)


      
        

In [2]:
def define_winners(string):
    if string=='':
        return [np.nan]*4
    
    elif '1.' in string:
        spots=string.split('\n2.')
        win1 = spots[0].split(' - ')
        if len(win1)>2:
            price1=win1[-1]
            win1=win1[0] +' - '+ win1[1]
        else:
            price1=win1[-1]
            win1=win1[0]
        try:
            win2=spots[1].split(' - ')
            if len(win2)>2:
                price2=win2[-1]
                win2=win2[0] +' - '+ win2[1] 
            else:
                price2=win2[-1]
                win2=win2[0]
        except:
            win2=np.nan
            price2=np.nan
            
        return [win1,price1,win2,price2]

def crawler_table_page(url,load_all):
    logging.info(f'......Preparing page for crawl: {url}; loading all: {load_all}')
    
    customOption = Options()
    customOption.add_argument("--headless")
    driver = webdriver.Chrome(executable_path = r'/users/bilyanavencheva/Applications/chromedriver'
                                  , options = customOption)

    driver.get(url)
    string='//*[@id="pub-menu"]/li[2]/a'
    l=driver.find_element("xpath",string)
    driver.execute_script("arguments[0].click();", l)
    
    if load_all is True:
        while True:
            try:
                time.sleep(5)
                ActionChains(driver).move_to_element(driver.find_element(By.XPATH,"//*[@id='more']")).perform()

                if driver.find_element(By.XPATH,"//*[@id='more']").is_enabled():
                    driver.find_element(By.XPATH,"//*[@id='more']").click()
                else:
                    driver.refresh()
            except Exception as e:
                print(f'{e}')
                break
            continue  
    else:
        pass
    
    time.sleep(5)
    
    logging.info(f'......Start of crawl: {url}; loading all: {load_all}')
    
    x_path='//*[@data-id]'
    all_auctions=driver.find_elements(By.XPATH,x_path)
    _dict_initial={}
    count=1
    for auction in all_auctions:
        cols = auction.find_elements(By.XPATH, "./*[name()='th' or name()='td']")
        temp = []
        for col in cols:
            if "Детайли" not in col.text:
                temp.append(col.text.strip())
            else:
                try:
                    temp.append(col.find_element(By.XPATH,'./a').get_attribute('href'))
                except Exception as e:
                    temp.append(e)

        _dict_initial[count] = temp
        
        count+=1
    
    driver.quit()
    
    columns=['Tърг№','Вид тръжна процедурa','ДГС/ДЛС','ДП','Обект№','Дървесен вид','Едра',
     'Средна', 'Дребна','Дърва за огрев','ОЗМ','Общо','Начална цена','Втора дата','Класиране','url']  #Втора дата - Дата на провеждане
    
    assert len(columns) == len(_dict_initial[1])
    
    list_all=[]
    for k,v in _dict_initial.items():
        _temp = {}
        for i, column in enumerate(v):
            _temp[columns[i]]=column.strip()
            
        _temp['ДП']=dp_name_dict[dp_name]
        _temp['други коментари'] = np.nan
        _temp['Първо място']=define_winners(string=_temp['Класиране'])[0]
        _temp['Първа цена']=define_winners(string=_temp['Класиране'])[1]
        _temp['Второ място']=define_winners(string=_temp['Класиране'])[2]
        _temp['Втора цена']=define_winners(string=_temp['Класиране'])[3]
        
        list_all.append(_temp)
        
    #this script creates duplicate entries for some reason, we need to drop them
    list_all_new = [dict(t) for t in {tuple(d.items()) for d in list_all}]
    
    json.dump(list_all, open(f"exports/archive/export_{GLOBAL_NAME}_{datetime.now()}.json","w"))
    logging.info(f'...... # of auctions crawled: {len(list_all_new)}')
    
    return list_all_new

def combine_all_table_data(PATH,type_,GLOBAL_NAME):
    latest_file, data = get_latest_file(PATH=PATH,type_=type_,aux=f'export_{GLOBAL_NAME}')
    df=pd.DataFrame(data)
    
    latest_file,data =get_latest_file(PATH=PATH,type_=type_,aux=f'export_detailed_{GLOBAL_NAME}')
    detailed_df=pd.DataFrame(data)
    
        
    for col in ['Tърг№','ДГС/ДЛС','Обект№','url']:
        for _df_ in [df,detailed_df]:
            _df_[col] = _df_[col].apply(lambda x:x.strip())
        
    data=df.merge(detailed_df, how='left',on=['Tърг№','ДГС/ДЛС','Обект№','url'])
    
    data['dumm'] = data['предмет'].apply(lambda x: 0 if x in ['Продажба на прогнозни количества'
                                                              ,'Отдаване на дървесина на корен'
                                                              ,'Продажба на действително добити количества дървесина от склад'] else 1)
    
    if len(data[data['dumm']==1]) != 0:
        print("ERROR: some columns were not matched")
        print(data[data['dumm']==1]['url'])
    
        
    return data



In [3]:



dp_name='sidp'
GLOBAL_NAME = GLOBAL_NAME_DICT[dp_name]

allowed_domains = ['sale.uslugi.io']
start_urls = [start_urls_dict[dp_name]]
base_url = start_urls_dict[dp_name]

# spider Class
class WoodSpider(scrapy.Spider):
    name = 'spider'
    allowed_domains = allowed_domains
    start_urls = start_urls
    base_url = base_url
    
    custom_settings = {
        'FEEDS': {
            f'exports/archive/export_detailed_{GLOBAL_NAME}_{datetime.now()}.json': {
                'format': 'json',
                'overwrite': True
            }
        }
    }
    
    
    def parse(self, response):
        logging.info(f'Start Spider Crawl for {start_urls}')
        print(f'Start Spider Crawl for {start_urls}')
        
        if len(extracted_urls)>0:
            logging.info(f'Extracting data from #: {len(extracted_urls)} files')
            
            for auction_url in extracted_urls:
                yield scrapy.Request(auction_url, callback=self.parse_auction)

        else:
            pass
    
    def parse_auction(self,response):
        print('Parse auction....', response.url) 
        logging.info(f'Parse auction.... {response.url}')
        _dict = {}
        _dict['url'] = response.url
        x_path="//*[@class='auct-title']/h4/text()"
        contents=response.xpath(x_path).extract()
        if len(contents)!=1:
            logging.warn(f'Error, invalid input {response}: {x_path} ---> {contents}')
            print(f'Error, invalid input {response}: {x_path} ---> {contents}')
        
        _dict['Tърг№'] = contents[0].split(' - ')[0].split(' ')[-1]
        _dict['ДГС/ДЛС'] = contents[0].split(' - ')[-1]
        
        x_path="//*[@class='auct-title']/h5/text()"
        contents=response.xpath(x_path).extract()
        
        if len(contents)!=1:
            logging.warn(f'Error, invalid input {response}: {x_path} ---> {contents}')
            print(f'Error, invalid input {response}: {x_path} ---> {contents}')
        _dict['Обект№'] = contents[0].split('No.')[-1].strip()
        
        x_path="//*[@class='auct-title']/p/text()"
        contents=response.xpath(x_path).extract()
        try:
            _dict['Първа дата'] = contents[0].split('Дата на публикуване:')[-1].strip()
        except:
            _dict['Първа дата'] = np.nan
            logging.warning('[Дата на публикуване] CANNOT be extracted')            
        
        x_path="//*[@class='info auct-type']/p/text()"
        contents=response.xpath(x_path).extract()
        if len(contents)!=2:
            logging.warn(f'Error, invalid input {response}: {x_path} ---> {contents}')
            print(f'Error, invalid input {response}: {x_path} ---> {contents}')
            
        _dict['вид търг'] = contents[0]
        try:
            _dict['предмет'] = contents[1]
        except:
            pass
        

        x_path= "//*[@class='doc-title col-sm-6']/div/p/a"
        for doc in response.xpath(x_path):
            name = doc.xpath('text()').extract()[0].lower()
            if 'protokol' in  name or 'protocol' in name or 'протокол' in name:
                url=doc.xpath('@href').extract()[0]
                _dict['Протокол'] = url
                
                competitors = extract_competitors_table(url,path = 'exports/protocols')
                _dict['брой участници (споменати)'] = competitors[0]
                _dict['участници (извлечени)'] = np.nan #str(competitors[1])
                _dict['брой участници (извлечени)'] = np.nan #len(competitors[1])
                _dict['страници протокол'] = competitors[2]
        yield _dict


In [4]:
# import tkinter
# import camelot
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.CRITICAL)

# name = '/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood/table_format/exports/protocols/Protokol+8-1-2023.pdf'

# abc = camelot.read_pdf(name)   #address of file location
 
# print(abc[0].df)
#reads table from pdf file
# df = wrapper.read_pdf(name,pages="all") #address of pdf file
# print(tabulate(df))
#from ctypes.util import find_library
#find_library("gs")

In [5]:
crawl_page=True
load_all=True
crawl_individual_url=True

if crawl_page is True:
    list_all = crawler_table_page(url=start_urls_dict[dp_name], load_all=load_all)
else: 
    pass

if crawl_individual_url == True:
    latest_file, data =get_latest_file(PATH=os.path.join(os.getcwd(),'exports/archive'),type_='json',aux=f'export_{GLOBAL_NAME}')
    extracted_urls = [k['url'] for k in data]
    
    process = CrawlerProcess()
    process.crawl(WoodSpider)
    process.start()
    
    data = combine_all_table_data(PATH=os.path.join(os.getcwd(),'exports/archive')
                                        ,type_='json'
                                        ,GLOBAL_NAME=GLOBAL_NAME)
    
    clean_data = transform_clean_data(data, GLOBAL_NAME=GLOBAL_NAME)
    


<ipython-input-2-a1fb0026ea09>:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r'/users/bilyanavencheva/Applications/chromedriver'


Message: element not interactable: https://sale.uslugi.io/sidp# has no size and location
  (Session info: headless chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x00000001051a4f38 chromedriver + 4910904
1   chromedriver                        0x0000000105124a03 chromedriver + 4385283
2   chromedriver                        0x0000000104d69747 chromedriver + 472903
3   chromedriver                        0x0000000104d6dd28 chromedriver + 490792
4   chromedriver                        0x0000000104d6d9f6 chromedriver + 489974
5   chromedriver                        0x0000000104d6ddfc chromedriver + 491004
6   chromedriver                        0x0000000104db6402 chromedriver + 787458
7   chromedriver                        0x0000000104db5627 chromedriver + 783911
8   chromedriver                        0x0000000104df9ca0 chromedriver + 1064096
9   chromedriver                        0x0000000104dd4112 chromedriver + 909586
10  chromedriver                     

2022-12-22 16:21:21 [py.warnings] WARNING: /opt/anaconda3/lib/python3.8/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2022-12-22 16:21:21 [scrapy.extensions.telnet] INFO: Telnet Password: ac39af739cfdfafb
2022-12-22 16:21:21 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2022-12-22 16:21:21 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2022-12-22 16:21:23 [filelock] DEBUG: Attempting to ac

Start Spider Crawl for ['https://sale.uslugi.io/sidp']
Parse auction.... https://sale.uslugi.io/sidp/5279


2022-12-22 16:21:23 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5287


...

2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5085
2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5088
2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5220
2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5092
2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5093
2022-12-22 16:31:58 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5222


Parse auction.... https://sale.uslugi.io/sidp/5085
Parse auction.... https://sale.uslugi.io/sidp/5088
Parse auction.... https://sale.uslugi.io/sidp/5220
Parse auction.... https://sale.uslugi.io/sidp/5092
Parse auction.... https://sale.uslugi.io/sidp/5093
Parse auction.... https://sale.uslugi.io/sidp/5222


2022-12-22 16:31:59 [root] INFO: Parse auction.... https://sale.uslugi.io/sidp/5221


Parse auction.... https://sale.uslugi.io/sidp/5221


2022-12-22 16:32:00 [scrapy.extensions.feedexport] INFO: Stored json feed (978 items) in: exports/archive/export_detailed_SIDP-SHUMEN_2022-12-22 16:03:17.741684.json
2022-12-22 16:32:00 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 304304,
 'downloader/request_count': 979,
 'downloader/request_method_count/GET': 979,
 'downloader/response_bytes': 2628073,
 'downloader/response_count': 979,
 'downloader/response_status_count/200': 979,
 'elapsed_time_seconds': 638.608819,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 12, 22, 14, 32, 0, 38470),
 'httpcompression/response_bytes': 9261483,
 'httpcompression/response_count': 979,
 'item_scraped_count': 978,
 'log_count/DEBUG': 2,
 'log_count/INFO': 996,
 'log_count/WARNING': 2551,
 'memusage/max': 356409344,
 'memusage/startup': 207982592,
 'request_depth_max': 1,
 'response_received_count': 979,
 'scheduler/dequeued': 979,
 'schedule

In [14]:
class TestsCleanData(object):
    
    def __init__(self,data_frame):
        self.clean_data=data_frame
    
    def to_zero(self, value):
        if isinstance(value,str):
            return 0
        else: 
            return value
    
    def run(self):
        clean_data = self.clean_data.copy()
        if len(clean_data['url'])!= len(clean_data['url'].unique()):
            print('Repeating entries for [url]')

        if np.max(clean_data['%увеличение'])>=2:
            print(f'Extreme values for  [%увеличение]: {len(clean_data[clean_data["%увеличение"]>=2])}')    

        if np.sum(clean_data.apply(lambda x: 1 if x['Договорена цена (лв.)'] < x['Втора цена'] else 0,axis=1)) != 0:
            print('Instances where [Договорена цена (лв.)] < [Втора цена]')  

        if np.sum(clean_data.apply(lambda x: 1 if x['Договорена цена (лв.)'] < x['Начална цена'] else 0,axis=1)) != 0:
            print('Instances where [Договорена цена (лв.)] < [Начална цена]')  

        if np.sum(clean_data.apply(lambda x: 1 if (np.isnan(self.to_zero(x['Първо място']))==False \
                                          and np.isnan(self.to_zero(x['Договорена цена (лв.)']))==True) \
                                   or (np.isnan(self.to_zero(x['Първо място']))==True \
                                       and np.isnan(self.to_zero(x['Договорена цена (лв.)']))==False) else 0,axis=1)):
            print('Instances with issues in [Първо място] ~ [Договорена цена (лв.)]')          

cd = TestsCleanData(data_frame=clean_data)
cd.run()
    


Extreme values for  [%увеличение]: 7
